To use this notebook, install the necessary packages by running the following command:  
pip install numpy matplotlib nnfs

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nnfs 
import math
from nnfs.datasets import spiral_data 
from nnfs.datasets import vertical_data

In [2]:
nnfs.init()

In [21]:
# Dense layer
class Layer_Dense:
    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
    # Forward pass
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases
        
    # Backward pass
    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)

In [22]:
# ReLU activation layer
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)
        
    # Backward pass
    def backward(self, dvalues):
        # Since we need to modify the original variable, we'll make a copy first
        self.dinputs = dvalues.copy()
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

In [5]:
# Softmax activation layer
class Activation_Softmax:
    # Forward pass
    def forward(self, input):
        # Get unnormalized probabilities
        exp_values = np.exp(input - np.max(input, axis=1, keepdims=True))
        
        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        
        self.output = probabilities

In [6]:
# Common loss class
class Loss:
    # Calculates the data and regularization losses 
    # given model output and ground truth values 
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        
        # Calculate mean loss
        data_loss = np.mean(sample_losses)
        
        # Return loss
        return data_loss

In [7]:
# Cross entropy loss
class Loss_CategoricalCrossentropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = len(y_pred)
        
        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        
        # Probabilities for target values - only if categorical labels
        # For sparse labels (y_true is a 1-D array of ints like [0,2,1]) this picks, for each sample i, the predicted probability for the correct class: y_pred_clipped[i, y_true[i]].
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
            
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
            
        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

In [8]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)

In [9]:
# Create Dense layer with 2 input features and 3 output values 
dense1 = Layer_Dense(2, 3)
print(dense1.weights)
print(' ')

# Create ReLU activation (to be used with dense layer)
activation1 = Activation_ReLU()

# Create second Dense layer with 3 input features (as we take output 
# of previous layer here) and 3 output values (output values) 
dense2 = Layer_Dense(3, 3)
print(dense2.weights)

# Create Softmax activation (to be used with Dense layer)
activation2 = Activation_Softmax()

# Create loss function 
loss_function = Loss_CategoricalCrossentropy()

[[-0.01306527  0.01658131 -0.00118164]
 [-0.00680178  0.00666383 -0.0046072 ]]
 
[[-0.01334258 -0.01346717  0.00693773]
 [-0.00159573 -0.00133702  0.01077744]
 [-0.01126826 -0.00730678 -0.0038488 ]]


In [10]:
# Make a forward pass of our training data through this layer
dense1.forward(X)

# Make a forward pass through activation function 
# it takes the output of first dense layer here 
activation1.forward(dense1.output)

# Make a forward pass through second Dense layer 
# it takes outputs of activation function of first layer as inputs 
dense2.forward(activation1.output)

# Make a forward pass through activation function 
# it takes the output of second dense layer here 
activation2.forward(dense2.output)

print(activation2.output[:5])  # Print first 5 outputs
print(' ')

# Perform a forward pass through loss function 
# it takes the output of second dense layer here and returns loss
loss = loss_function.calculate(activation2.output, y)

# print loss value
print('Loss:', loss)

[[0.33333334 0.33333334 0.33333334]
 [0.3333332  0.3333332  0.33333364]
 [0.3333329  0.33333293 0.3333342 ]
 [0.3333326  0.33333263 0.33333477]
 [0.33333233 0.3333324  0.33333528]]
 
Loss: 1.0986104


In [11]:
# Calculate accuracy from output of activation2 and targets 
# calculate values along first axis 
predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
   y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)
# Print accuracy
print('acc:', accuracy)

acc: 0.34
